In [ ]:
import torch
import torchvision
import os
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# batch_size = 4

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
directory = 'processedDataset/'
class StockDataset(Dataset):
    def __init__(self):
        self.data = []
        i = 0
        tempData = os.listdir(directory)
        while i < 640:
            f = os.path.join(directory, tempData[i])
            if torch.any(torch.isnan(torch.load(f))):
                print(f + " contains nan")
                continue
            else:
                label = int(torch.load(f)[0][0] > torch.load(f)[1][0])
                self.data.append([torch.load(f).float(), label])
                i += 1
                # if filename.split('-')[2].replace(".pt","") == '1':
                #     Y.append(torch.tensor([0,1]).float())
                # else:
                #     Y.append(torch.tensor([1,0]).float())
            

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i][0], self.data[i][1]

In [ ]:
class TestStockDataset(Dataset):
    def __init__(self):
        self.data = []
        i = 0
        tempData = os.listdir(directory)
        tempData.reverse()
        while i < 64:
            f = os.path.join(directory, tempData[i])
            if torch.any(torch.isnan(torch.load(f))):
                print(f + " contains nan")
                continue
            else:
                label = int(torch.load(f)[0][0] > torch.load(f)[1][0])
                self.data.append([torch.load(f).float(), label])
                i += 1
                # if filename.split('-')[2].replace(".pt","") == '1':
                #     Y.append(torch.tensor([0,1]).float())
                # else:
                #     Y.append(torch.tensor([1,0]).float())
            

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i][0], self.data[i][1]

In [ ]:
batch_size = 64
stockDataset = StockDataset()
testDataset = TestStockDataset()
trainloader = torch.utils.data.DataLoader(stockDataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testDataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(104 * 22, 2)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 2)
    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = torch.nn.functional.normalize(x, p=2.0, dim = 1)
        x = self.fc1(x)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from cProfile import run
import string


for epoch in range(100001):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    if epoch % 1000 == 0:
        print("epoch: " + str(epoch) + " loss: " + str(running_loss/10))

print('Finished Training')

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
dataiter = iter(testloader)
stocks, labels = dataiter.next()

In [ ]:
outputs = net(stocks)

In [ ]:
_, predicted = torch.max(outputs, 1)
print(predicted)
print(labels)

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')